In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import (Conv2D,LeakyReLU,Dense,Flatten,
                                     Dropout,BatchNormalization,UpSampling2D,
                                     GlobalMaxPooling2D,Reshape,Conv2DTranspose)
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model, callbacks
from tensorflow.keras.preprocessing.image import array_to_img
import os


In [ ]:
latent_dim= 128
batch_size = 64
(x_train,_),(x_test,_) = tf.keras.datasets.fashion_mnist.load_data()
all_images = np.concatenate((x_train,x_test), axis=0)
all_images = all_images.astype("float32") / 255.0
all_images = np.reshape(all_images,(-1,28,28,1))
dataset = tf.data.Dataset.from_tensor_slices(all_images)
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size).prefetch(32)


4422102/4422102 [==============================] - 0s 0us/step


In [ ]:
class Create_Models():
    def __init__(self,latent_dim):
        
        self.latent_size = latent_dim
        
    def build_disc(self):
        model= Sequential()
        model.add(Conv2D(32,3,padding='same', input_shape=(28,28,1)))
        model.add(LeakyReLU(alpha=0.25))
        model.add(Dropout(0.5))

        model.add(Conv2D(64,3,padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.25))
        model.add(Dropout(0.5))
         
        model.add(Conv2D(128,3,strides=2,padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.25))
        model.add(Dropout(0.5))
        
        model.add(Conv2D(256,3,strides=1,padding='same'))
        model.add(BatchNormalization())
        model.add(LeakyReLU(alpha=0.25))
        model.add(Flatten())
        
        model.add(Dropout(0.5))
        
#         model.add(GlobalMaxPooling2D())
    
        model.add(Dense(1, activation='sigmoid')) 
        
        return model  
    
    def build_gen(self):
        
        model = tf.keras.models.Sequential()

        model.add(tf.keras.layers.Dense(7*7*128,input_dim=self.latent_size))
        model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
        model.add(tf.keras.layers.Reshape((7,7,128)))

        #128 14*14
        model.add(tf.keras.layers.Conv2DTranspose(128,4,strides=2,padding='same',kernel_initializer='glorot_normal'))
        model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
        model.add(tf.keras.layers.BatchNormalization())

        #128* 128*28*28
        model.add(tf.keras.layers.Conv2DTranspose(128,4,strides=2,padding='same',kernel_initializer='glorot_normal'))
        model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
        model.add(tf.keras.layers.BatchNormalization())
        
        model.add(tf.keras.layers.Conv2DTranspose(64,2,strides=1,padding='same',kernel_initializer='glorot_normal'))
        model.add(tf.keras.layers.LeakyReLU(alpha=0.2))
        model.add(tf.keras.layers.BatchNormalization())
        
        model.add(tf.keras.layers.Conv2D(1,7,padding='same',activation='tanh',kernel_initializer='glorot_normal'))

        return model

    

In [ ]:
creat_model = Create_Models(latent_dim)
generator= creat_model.build_gen()
discriminator= creat_model.build_disc()

In [ ]:
class DCGAN(Model):
    def __init__(self, discriminator, generator, latent_dim,*args,**kwargs):
        super(DCGAN, self).__init__(*args,**kwargs)
        self.discriminator =discriminator
        self.generator =generator        
        self.latent_dim =latent_dim     
        
    def compileM(self,d_optimizer,g_optimizer, loss_fn,*args,**kwargs):
        super(DCGAN,self).compile(*args,**kwargs)
        self.d_optimizer =d_optimizer
        self.g_optimizer =g_optimizer        
        self.loss_fn =loss_fn     
    
    def train_step(self, real_images):
        
        batch_size= tf.shape(real_images)[0]
        random_latent_vects = tf.random.normal(shape=(batch_size,self.latent_dim))
        gen_images = self.generator(random_latent_vects)
        combind_images= tf.concat([gen_images, real_images],axis=0)
        
        
        # train discriminator
        
        labels = tf.concat([tf.ones((batch_size,1)), tf.zeros((batch_size,1))],axis=0)
        labels += 0.04* tf.random.uniform(tf.shape(labels))
        
        with tf.GradientTape() as tape:
            predictions = self.discriminator(combind_images)
            d_loss = self.loss_fn(labels, predictions)
            
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )
         
        # train generator
        
        random_latent_vects = tf.random.normal(shape=(batch_size,self.latent_dim))
        fake_labels = tf.zeros((batch_size, 1))
                
        with tf.GradientTape() as tape:
            predictions = self.discriminator(self.generator(random_latent_vects))
            g_loss = self.loss_fn(fake_labels,predictions)
            

        grads = tape.gradient(g_loss,self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(
            zip(grads,self.generator.trainable_weights)
        )
        return {"d_loss":d_loss,"g_loss":g_loss}


In [ ]:
class DCGAN_Monitor(callbacks.Callback):
    def __init__(self,latent_dim,num_image=3):
        self.latent_dim = latent_dim
        self.num_image= num_image
    
    def on_epoch_end(self, epoch, logs=None):
        random_latent_vects =tf.random.normal(shape=(16, self.latent_dim))
        gen_images = self.model.generator(random_latent_vects)
        generator_images = gen_images.numpy()
        plt.close('all')
        fig,axs = plt.subplots(4,4,figsize=(4,4),sharey=True,sharex=True)
        for i in range(self.num_image):
          img= array_to_img(generator_images[i])

        c=0
        for i in range(4):
            for j in range(4):
                axs[i,j].imshow(gen_images[c,:,:,0],cmap='gray')
                axs[i,j].axis('off')
                c+=1

        fig.show()
                

In [ ]:
dcgan = DCGAN(discriminator=discriminator,generator=generator,latent_dim=latent_dim)
dcgan.compileM(
    d_optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001,beta_1=0.5),
    g_optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001,beta_1=0.5),
    loss_fn=tf.keras.losses.BinaryCrossentropy(from_logits=True)
)
dcgan.fit(dataset,epochs= 200 ,callbacks=[DCGAN_Monitor(latent_dim=latent_dim)])

Epoch 1/200


/usr/local/lib/python3.9/dist-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1094/1094 [==============================] - 84s 62ms/step - d_loss: 0.2714 - g_loss: 2.4431
Epoch 2/200
1094/1094 [==============================] - 67s 61ms/step - d_loss: 0.1780 - g_loss: 3.1339
Epoch 3/200
1094/1094 [==============================] - 68s 62ms/step - d_loss: 0.2243 - g_loss: 2.8309
Epoch 4/200
1094/1094 [==============================] - 66s 61ms/step - d_loss: 0.2518 - g_loss: 2.4634
Epoch 5/200
1094/1094 [==============================] - 66s 61ms/step - d_loss: 0.2259 - g_loss: 2.5767
Epoch 6/200
1094/1094 [==============================] - 67s 61ms/step - d_loss: 0.2280 - g_loss: 2.6991
Epoch 7/200
1094/1094 [==============================] - 67s 61ms/step - d_loss: 0.2233 - g_loss: 2.7248
Epoch 8/200
1094/1094 [==============================] - 66s 61ms/step - d_loss: 0.2262 - g_loss: 2.7175
Epoch 9/200
1094/1094 [==============================] - 66s 61ms/step - d_loss: 0.2310 - g_loss: 2.6932
Epoch 10/200
1094/1094 [==============================] - 66s 61ms/

In [ ]:
imgs = generator.predict(tf.random.normal((16,128,1)))
plt.close('all')
fig,axs = plt.subplots(4,4,figsize=(4,4),sharey=True,sharex=True)

c=0
for i in range(4):
    for j in range(4):
        axs[i,j].imshow(imgs[(i+1)*(j+1)-1],cmap='gray')
        axs[i,j].axis('off')
        c+=1
fig.show()